# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform google-cloud-trace --upgrade

### Initialize Vertex AI Client

In [ ]:
# Initialize the Vertex AI client
import json
import vertexai
import sys
sys.path.append('/content/agent-engine-test/app/')
LOCATION = "us-central1"
PROJECT_ID = "YOUR_PROJECT_ID"
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
#AGENT_ENGINE_ID = None
AGENT_ENGINE_ID = "0000000000000000000"
if AGENT_ENGINE_ID != None:
    REASONING_ENGINE_ID = f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{AGENT_ENGINE_ID}"
else:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
client = vertexai.Client(
    location=LOCATION,
)

## Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [ ]:
print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

Generate test session

In [ ]:
session = await remote_agent_engine.async_create_session(user_id="test")
session

Send test query #1

In [ ]:
async for event in remote_agent_engine.async_stream_query(
    message="닭고기 메뉴가 있을까요 ?", user_id="test", session_id = session['id']
):
    print(event)

Send test query #2

In [ ]:
async for event in remote_agent_engine.async_stream_query(
    message="저는 채식주의자예요.", user_id="test", session_id = session['id']
):
    print(event)

Invoke custom function

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

Manually create memory (By default, memory is not automatically generated)

Refer below for detail control of memory

https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/agents/agent_engine/memory_bank/get_started_with_memory_bank.ipynb

In [ ]:
session = await remote_agent_engine.async_get_session(user_id="test", session_id=session['id'])
await remote_agent_engine.async_add_session_to_memory(session=session)

Search memory using similarity search by query

In [ ]:
await remote_agent_engine.async_search_memory(user_id="test", query="San Francisco")

In [ ]:
pager = client.agent_engines.memories.list(name=REASONING_ENGINE_ID)
for i, page in enumerate(pager):
  print(f"========== Memory {i} =========")
  print(f"Expire: {page.expire_time}")
  print(f"TTL: {page.ttl}")
  print(f"Revision Expire: {page.revision_expire_time}")
  print(f"Revision TTL: {page.revision_ttl}")
  print(f"Disable memory revisions: {page.disable_memory_revisions}")
  print(f"Create time: {page.create_time}")
  print(f"Description: {page.description}")
  print(f"Display name: {page.display_name}")
  print(f"Fact: {page.fact}")
  print(f"Name: {page.name}")
  print(f"Scope: {page.scope}")
  print(f"Update time: {page.update_time}")
  print("\n")


Get Trace data (https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb)

In [ ]:
from google.cloud import trace_v1 as trace
client = trace.TraceServiceClient()
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            #filter="trace_id:d5640adfabcb9f9678fb37f63471b0ae",
            filter="root:invocation",
            order_by="start desc" #Decending order
        )
    )
]
result

In [ ]:
client.get_trace(
    request=trace.types.GetTraceRequest(
            project_id=PROJECT_ID,
            trace_id = result[0].trace_id
        )
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

Refer following link for agent engine local testing

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/memory/get_started_with_memory_bank_adk.ipynb

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
import sys
sys.path.insert(0, '../')
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

Delete agent

In [ ]:
client = vertexai.Client(
    location=LOCATION,
)
client.agent_engines.delete(
    name=REASONING_ENGINE_ID,
    force=True, # Optional, if the agent has resources (e.g. sessions, memory)
)